# Section 2

## Topics
<ul>
<li>TensorFlow Sessions</li>
<li>Feeding data to TF Graph</li>
<li>Graph Outputs</li>
<li>Loss Functions</li>
<li>Optimizers</li>
</ul>
<br/>
<i>Some code samples and quotes from tensorflow.org documentation.</i>
<br/>
## TensorFlow Sessions
"A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated."

1) Create the session <br/>
2) Initialize variables if you need to (we don't for now and will return to this later). <br/>
3) Run one or more operations in your graph. <br/>

In [1]:
import tensorflow as tf

# Build a graph.
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

# STYLE 1: Keep a handle to the tf.Session() object and keep it forever.
# Launch the graph in a session.
sess = tf.Session()
# Evaluate the tensor `c`.
print('Style 1 output: ' + str(sess.run(c)))
# Remember to close the session when you are done with it.
sess.close()

# STYLE 2: Do all your computation within a python with block.
with tf.Session() as sess:
    print('Style 2 output: ' + str(sess.run(c)))

Style 1 output: 30.0
Style 2 output: 30.0


## Feeding Data to TensorFlow Graph
Now we will create a graph that accepts inputs and pass these into our graph with a TensorFlow Session.<br/>

Data flows into a TF Graph through placeholders.

In [2]:
# Create two placeholders that take any sized arrays.
inputs1 = tf.placeholder(tf.float32, shape=(None), name='inputs1')
inputs2 = tf.placeholder(tf.float32, shape=(None), name='inputs2')

# Create a graph that multiplies each of these element-wise.
outputs = tf.multiply(inputs1, inputs2)

# Before we just used constants but now we need our own data to pass in.
# Let's create some random data values using numpy.
import numpy as np

samples = 5
inputs1_data = np.random.rand(samples)
inputs2_data = np.random.rand(samples)

# We pass this data into the graph through a feed_dict (a.k.a. feed dictionary)
# feed dictionaries map placeholders to the data you are passing to them
feed_dict={inputs1: inputs1_data, inputs2: inputs2_data}

# We then start a session and run the output operation in our new graph with the feed_dict that will pass data into it.
with tf.Session() as sess:
    out = sess.run(outputs, feed_dict)
    print('Style 1 output: ' + str(out))
    
# Or more cleanly (and more commonly used)
with tf.Session() as sess:
    out = sess.run(outputs, feed_dict={inputs1: inputs1_data, inputs2: inputs2_data})
    print('Style 2 output: ' + str(out))
    
# Congrats!  You just fed your own data into a TensorFlow Graph and computed some output

Style 1 output: [ 0.14236929  0.29745367  0.0892252   0.53928107  0.11020289]
Style 2 output: [ 0.14236929  0.29745367  0.0892252   0.53928107  0.11020289]


## Graph Outputs
So far every graph we have looked at has a single output.  TensorFlow is much more powerful than that. <br/>

You can have multiple outputs

In [3]:
inputs = tf.placeholder(tf.float32, (None), name='inputs')
output1 = tf.multiply(inputs, 2)
output2 = tf.add(inputs, 5)
num_samples = 5

with tf.Session() as sess:
    input_data = np.random.rand(num_samples)
    [out1, out2] = sess.run([output1, output2], feed_dict={inputs: input_data})
    print('\nInput Data:\n' + str(input_data) + '\n')
    print('Result of multiplying by 2:\n' + str(out1) + '\n')
    print('Result of adding 5:\n' + str(out2) + '\n')


Input Data:
[ 0.19161177  0.84103391  0.14637289  0.06686248  0.37977766]

Result of multiplying by 2:
[ 0.38322353  1.68206787  0.29274577  0.13372496  0.75955534]

Result of adding 5:
[ 5.19161177  5.84103394  5.1463728   5.06686258  5.37977791]



In [4]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
# Show the current graph
show_graph(tf.get_default_graph().as_graph_def())

You can also grab any operation in the 'middle' of your graph and run it.

In [5]:
inputs = tf.constant(10)
multiplied = tf.multiply(inputs, 2)
added = tf.add(multiplied, 3)
squared = tf.pow(added, 2)
output = tf.subtract(squared, 5)

with tf.Session() as sess:
    print('10 * 2 = ' + str(sess.run(multiplied)))
    print('10 * 2 + 3 = ' + str(sess.run(added)))
    print('(10 * 2 + 3) ^ 2 = ' + str(sess.run(squared)))
    print('(10 * 2 + 3) ^ 2 - 5 = ' + str(sess.run(output)))
    
# Mix-and-match any set of operations in your graph that you want to run.
print('\nMix-and-match: ' + str(tf.Session().run([multiplied, output])))

10 * 2 = 20
10 * 2 + 3 = 23
(10 * 2 + 3) ^ 2 = 529
(10 * 2 + 3) ^ 2 - 5 = 524

Mix-and-match: [20, 524]


As your graphs get more complicated you will have all sorts of ops that you pass to sess.run(...)
<ul>
<li>Output values</li>
<li>Metrics (ex: accuracy)</li>
<li>Learning rate</li>
<li>etc.</li>
</ul>

## Loss Functions
Moving past simple, hard-coded graphs we want to be able to train a graph with a dataset.  Many of your models will be trained by specifying the structure of the network, a loss function, a dataset, and an optimizer that updates the model parameters based on that dataset.

Tensorflow has many loss functions built-in to make them easy to use.<br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/losses'>TensorFlow Loss Functions</a>

In [6]:
# Using a TF Loss Function
input_dim = 2
output_dim = 1
inputs = tf.placeholder(tf.float32, (input_dim, 1))
labels = tf.placeholder(tf.float32, (output_dim, 1))
# Create some random initial parameter values
weights = tf.Variable(tf.random_normal([output_dim, input_dim], stddev=0.35),
                      name="weights")
print(weights)

# y = Wx
logits = tf.matmul(weights, inputs)
print(logits)

# Create the loss function
loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
print(loss)

<tf.Variable 'weights:0' shape=(1, 2) dtype=float32_ref>
Tensor("MatMul:0", shape=(1, 1), dtype=float32)
Tensor("Reshape_2:0", shape=(1,), dtype=float32)


## Optimizer
We have a loss function now which tells us how well we are doing but no way of updating our weights to make our model any better.  Introduce an optimizer.  Optimizers make changes to the weights in your model in a direction that should improve the prediction for the next data point.  See <a href='http://ruder.io/optimizing-gradient-descent/'>this</a> blog post on different gradient based optimizers.

TensorFlow has several built-in optimizers you can use.<br/>
<a href='https://www.tensorflow.org/api_guides/python/train'>TensorFlow Optimizers</a>

Let's set up the momentum optimizer to work with our model.

In [10]:
optimizer = tf.train.MomentumOptimizer(0.1, 0.9) # 0.1 is the learning rate, 0.9 is the momentum term
minimizer = optimizer.minimize(loss)

# Toy Dataset
samples = 20
input_data = np.random.rand(input_dim, 1)
label_data = np.ones((output_dim, 1))
print(input_data, " jn ", label_data)

# Create out session
sess = tf.Session()

# Now we have variables to initialize
sess.run(tf.global_variables_initializer())

_, weight_out = sess.run([minimizer, weights], feed_dict={inputs: input_data, labels: label_data})
print(weight_out) 
sess.close()

[[ 0.49303384]
 [ 0.85338054]]  jn  [[ 1.]]
[[-0.05060205 -0.02471533]]
